################################################################################
#Licensed Materials - Property of IBM
#(C) Copyright IBM Corp. 2020
#US Government Users Restricted Rights - Use, duplication disclosure restricted
#by GSA ADP Schedule Contract with IBM Corp.
################################################################################

The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below.
Specifically, the Source Components and Sample Materials clause included in the License Information document for
Watson Studio Auto-generated Notebook applies to the auto-generated notebooks. 
By downloading, copying, accessing, or otherwise using the materials, you agree to the License Terms.
http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1

## IBM AutoAI Auto-Generated Notebook v1.11.10
**Note**: Notebook code generated using AutoAI will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
This pipeline is optimized for the original dataset.The pipeline may fail or produce sub-optimium results if used with different data.
For different data, please consider returning to AutoAI Experiments to generate a new pipeline.
Please read our documentation for more information: 
(Cloud Platform) https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .
(Cloud Pak For Data) https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/autoai-notebook.html .

Before modifying the pipeline or trying to re-fit the pipeline, consider: 
The notebook converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).
The known_values_list is passed by reference and populated with categorical values during fit of the preprocessing pipeline.Delete its members before re-fitting.

### Representing Pipeline from run: Pipeline_8 from run f0c1d8d59c0f41b68561d37564370658


### 1. Set Up

In [1]:

try:
    import autoai_libs
except Exception as e:
    import subprocess
    out = subprocess.check_output('pip install autoai-libs'.split(' '))
    for line in out.splitlines():
        print(line)
    import autoai_libs
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import inf, nan, dtype, mean
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]

b'Collecting autoai-libs'
b'  Downloading autoai_libs-1.12.4-48-cp36-cp36m-manylinux1_x86_64.whl (4.4 MB)'
b'Requirement already satisfied: scikit-learn<0.23.2,>=0.20.3 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from autoai-libs) (0.22.1)'
b'Requirement already satisfied: pandas<1.0.0,>=0.24.2 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from autoai-libs) (0.24.2)'
b'Requirement already satisfied: numpy>=1.16.4 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from autoai-libs) (1.16.6)'
b'Requirement already satisfied: scipy>=0.17.0 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from scikit-learn<0.23.2,>=0.20.3->autoai-libs) (1.3.2)'
b'Requirement already satisfied: joblib>=0.11 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages (from scikit-learn<0.23.2,>=0.20.3->autoai-libs) (0.14.1)'
b'Requirement already satisfied: python-dateutil>=2.5.0 in /opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-p

In [2]:
! cat TExtras

cat: TExtras: No such file or directory


In [3]:
# compose a decorator to assist pipeline instantiation via import of modules and installation of packages
def decorator_retries(func):
    def install_import_retry(*args, **kwargs):
        retries = 0
        successful = False
        failed_retries = 0
        while retries < 100 and failed_retries < 10 and not successful:
            retries += 1
            failed_retries += 1
            try:
                result = func(*args, **kwargs)
                successful = True
            except Exception as e:
                estr = str(e)
                if estr.startswith('name ') and estr.endswith(' is not defined'):
                    try:
                        import importlib
                        module_name = estr.split("'")[1]
                        module = importlib.import_module(module_name)
                        globals().update({module_name: module})
                        print('import successful for ' + module_name)
                        failed_retries -= 1
                    except Exception as import_failure:
                        print('import of ' + module_name + ' failed with: ' + str(import_failure))
                        import subprocess
                        print('attempting pip install of ' + module_name)
                        process = subprocess.Popen('pip install ' + module_name, shell=True)
                        process.wait()
                        try:
                            print('re-attempting import of ' + module_name)
                            module = importlib.import_module(module_name)
                            globals().update({module_name: module})
                            print('import successful for ' + module_name)
                            failed_retries -= 1
                        except Exception as import_or_installation_failure:
                            print('failure installing and/or importing ' + module_name + ' error was: ' + str(
                                import_or_installation_failure))
                            raise (ModuleNotFoundError('Missing package in environment for ' + module_name +
                                                       '? Try import and/or pip install manually?'))
                elif type(e) is AttributeError:
                    if 'module ' in estr and ' has no attribute ' in estr:
                        pieces = estr.split("'")
                        if len(pieces) == 5:
                            try:
                                import importlib
                                print('re-attempting import of ' + pieces[3] + ' from ' + pieces[1])
                                module = importlib.import_module('.' + pieces[3], pieces[1])
                                failed_retries -= 1
                            except:
                                print('failed attempt to import ' + pieces[3])
                                raise (e)
                        else:
                            raise (e)
                else:
                    raise (e)
        if successful:
            print('Pipeline successfully instantiated')
        else:
            raise (ModuleNotFoundError(
                'Remaining missing imports/packages in environment? Retry cell and/or try pip install manually?'))
        return result
    return install_import_retry


### 2. Compose Pipeline

In [4]:
# metadata necessary to replicate AutoAI scores with the pipeline
_input_metadata = {'target_label_name': 'Retention', 'learning_type': 'classification', 'run_uid': 'f0c1d8d59c0f41b68561d37564370658', 'pn': 'P8', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'f1', 'pos_label': None, 'random_state': 33, 'data_source': ''}

# define a function to compose the pipeline, and invoke it
@decorator_retries
def compose_pipeline():
    import numpy
    from numpy import nan, dtype, mean
    #
    # composing steps for toplevel Pipeline
    #
    _input_metadata = {'target_label_name': 'Retention', 'learning_type': 'classification', 'run_uid': 'f0c1d8d59c0f41b68561d37564370658', 'pn': 'P8', 'cv_num_folds': 3, 'holdout_fraction': 0.1, 'optimization_metric': 'f1', 'pos_label': None, 'random_state': 33, 'data_source': ''}
    steps = []
    #
    # composing steps for preprocessor Pipeline
    #
    preprocessor__input_metadata = None
    preprocessor_steps = []
    #
    # composing steps for preprocessor_features FeatureUnion
    #
    preprocessor_features_transformer_list = []
    #
    # composing steps for preprocessor_features_categorical Pipeline
    #
    preprocessor_features_categorical__input_metadata = None
    preprocessor_features_categorical_steps = []
    preprocessor_features_categorical_steps.append(('cat_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27])))
    preprocessor_features_categorical_steps.append(('cat_compress_strings', autoai_libs.transformers.exportable.CompressStrings(activate_flag=True, compress_type='hash', dtypes_list=['char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'int_num', 'int_num', 'int_num', 'int_num', 'int_num', 'char_str', 'char_str', 'char_str', 'char_str', 'char_str', 'int_num', 'char_str', 'char_str', 'int_num', 'int_num', 'char_str', 'char_str'], missing_values_reference_list=['', '-', '?', nan], misslist_list=[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])))
    preprocessor_features_categorical_steps.append(('cat_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_categorical_steps.append(('cat_unknown_replacer', autoai_libs.transformers.exportable.NumpyReplaceUnknownValues(filling_values=nan, filling_values_list=[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], known_values_list=[[243382710732531489693195870365903964183, 132302907946296948163998721551176913917], [285653138755766267379049297412757112339, 275921507576585114321456158174289189803, 295722777621721831987190764977671769241, 227310501169485414093933345603149554622, 110195480183711974516242104684569020453, 245907536313989839451450548589099059474, 29959270294701566056520519734183045334], [213368931744182528744866099603730755604, 187947410709490631541604360685754209828], [248552393553625911829275175950507086347, 196453806044956024215798110170812425269], [322857047585866530917354998729024057565, 78226537117366630030976571890067276293, 136547471614135744345948147656287155038], [95668235662009941600707143325543633137, 81116575116153187023093434037991966189], [248552393553625911829275175950507086347, 196453806044956024215798110170812425269], [248552393553625911829275175950507086347, 196453806044956024215798110170812425269], [248552393553625911829275175950507086347, 196453806044956024215798110170812425269], [780, 1975], [93600, 237000], [0, 1, 2], [0, 1, 2, 3, 4], [0, 1, 2, 3], [39775928476453518192143468584069194891, 312937356636936619578475450803013523595, 174681000509555248901248526300614640175], [248552393553625911829275175950507086347, 196453806044956024215798110170812425269], [261367234160207730710124948678089698558, 227892522217495654672057303313536575094, 28779897347380103346071804070587709992, 135084442495823429317223942504818887941, 8831443835998949901558691218505788153, 112730991179840432557095823906512738377], [286323945598653515978685547335610543390, 292807103544294102579548152187614479554, 144878164230350349449658835495233202991, 183937131109250236753371526229213969901], [16505481078797947184087459049083529932, 19381872465482661651387201646182552589], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [339914797681135161069053613952454376331, 328228002777967777108851274772865634799, 67173585239538071839587539288433045831], [212888346288348889924958930886266207537, 171101301351513372456262735919285431564, 132386532056094759114448777058593500755, 64510307834606397875652004196813071011], [0, 1, 2, 3], [0, 1, 2, 3, 4, 5, 6], [17281772905807082700562939188117103858, 132311617304685644166986907700846127015, 337976405248649039172369208992958740755, 12920311373468132796491748745244363656, 220823866455901090025550598112313225904, 180505624056785205984385937996497344173, 204439820912366850673907978134288456888, 193884206951017958907165796460746188292, 47384724961456585366399559105252481769, 279886702240868405111364510818153103729, 174467421861053187459382040560129201058, 216028245545433420174808712027585467922, 207778965532259063465251753924930347109, 100071657621190392961636984615741629142, 231146694314467319537985082736155923144, 35039104003445789395837328593719196253, 181280895118987245375232937139776936726, 220734857687564004773246030464356823140, 116841694356327817930391518890306196170, 138972469845409856787631377788996664479, 326928856373584558519269361208003921944, 134021079626367958950854800845082589277, 119110834086324578420949535624229002926, 24748827531422550278917507522818810384, 185734528398139087918734882309085910056, 297372218854232400820598901549200782221, 69549408309202364212146822135750659938, 134660214165210252565351825091728324396, 119770182185029238057233179624404789178, 236500324432145003513982363573227031523, 114311122962465730137395144657661934295, 1720017911291219204916874118921898875, 135600398801710525763011566747095663355, 70846155901416542898894785590730795361, 25608867485437277426991076499744121283, 159727422273452139181303440002327383679, 35928353751425220377404412709176531594, 76420881116409039774058566772965032004, 35091054447591088513711533648627074071, 82688778667621129133159808634079357582, 256215364898967517137696577920950949077, 248657403260076104384576435243307471924, 31290881666559732445420115378727930651, 329104880848179624402972826950996616483, 294982431882423559235597036839512558538, 329127422335589363299262808397301694093, 45796366042256341737184094599330328748, 252004254566053426001430868606859055269, 240564668192350280201091398528287496947, 239300502887507094417759547710929924130, 339382423386251335379909156441763860850, 306274798026049698754674672362106972414, 218773599468554609109044561891290884441, 66960039217726351722293790895427181134, 51376250805904524586728502108394542586, 42271695803300054194284527670446639120, 66015833561240425102488184993266333301, 97958193249312944214895502058977600266, 211989289431622221854810802263168793490, 156523825802198925354457472264827996068, 248935390122004079351768803368447769206, 195784113272022215859259655546468738703, 312934742346707744564826307041096819800, 234909242787271486880779945439209107777, 40822756829701798296834156041645029544, 181163317281570259118609963090831775764, 65854463757825850606464824964233235015, 65411223932020445940256852450264899265, 189830131246317827646747167762919375682, 222831180472238000361371739071057914259, 18313675031730253626390008597913439309, 214612759870264211594910256304139704550, 4736289755114884018090803676864665097, 284027545399100224804960288519569793877, 291079925102990117698638623897408492156, 177286290942243516482806090660792286722, 108466841473092277742317332510467319378, 69644525748586293583829845102726469931, 179771275640587060885486904982407251686, 227675983814006881621556991747694292052, 328636227788562795364033489989915934122, 177707718066266989763270181886556206908, 239664106605130623513873106717651591524, 193632073051356350060403232279661147754, 45801963356225029782285903223076332511, 293619177639267156338887703979663913830, 33857636394153860594096273092852066258, 252486071409039000216257558965697250580, 150009296282157300243723937567632112023, 261895702635012322618670287791661328633, 265844091122728204123183023271356080675, 338922190576601260391782658249772918990, 332530227579696939928316659100288072496, 295006362381996262398341237864338804389, 297158226273679599178065246610504596137, 286664860333807102344326963676777212773, 340041480336808234397653832782759632880, 251469622026818333629517718278674902380, 11596166966771371301712831911395343024, 339397463926867162822096374472867967742, 277490691368511505600708426937033535028, 156631732283550760558427320232790428296, 123350979273313908305173644975832639647, 173529420979897375284318237583786677066, 232807657405964239380734931994564430323, 76884739790720981739057536075013844006, 244308391327843309772372161021461356234, 167577208518908713986986542630355336786, 51104397927183410037618864854085087979, 120227558099482767679314314519869476962, 227306507003934666721972680409399505527, 31921349541262498345956411052580254391, 152662152584119829467183535876025105088, 153290631455062406289134131707786095180, 238252454301013771772476739927783236394, 318505167595503321233502966550699735101, 254088877810985795913429169141913338872, 166907270435803983120498288849521363376, 107855127919703283356422570999219335903, 317528026039591287958313125531099496178, 336539357518849815939016182011835451878, 314789509810715123691921924871107988724, 12294303937160961180757127127604108051, 140322358442612850053904137210115955293, 102053011397607698783411831973186713122, 136295362063807677684783956503993737546, 124594693483616277579498057240997756093, 320470962803031218650373846942761464858, 1019807582802610015379078603317322373, 197831920536627631528562604090955364269, 267267749241871724913747036526881824758, 195517137843310412084438519058507993770, 101691166024367715191072864897119219684, 167096495630474370416862919846863916694, 197353011193714326181543954809903063096, 158253089345086766149395257522781473841, 264430160686462553599126090313978775481, 127966120654367474954410292588714523644], [44196315322896597290292502757424465922, 299541491905309596459425207474481512849, 84930404695302323838977356069304917328, 93933606135191583877076205776863971778, 70986357964105645842605941046486920524, 29100642732153115018509383391590601675, 328748176432085826825247909159961688195, 247449615375995846643716445202733737804, 317252602033763261150504482963731639224, 312173900060238937270521904129323847731, 223798132513453761469470875660301839786, 95300428989802962770366978454830954305, 136592194982456402346452786321636698551, 262145702306492789121428997430250344889, 18167616761854635495214567335744092953, 164897847420850527505191271311954980477, 202109805494104726511873358199340169272, 76900096691716753194694726412438795012, 86946887994913847157665549413895404232, 23814184504637585797280731443900147111, 166193442674975922007755749062350582465, 189652655179744954980030450519584696968, 91719881352629408390057862635849729129, 27608926569021204910106444897289762726, 9274428055267899800877186455717105788, 223354670275451005457620867750119387651, 43052486412569332607413253590872511435, 191153072498291133524386212799009700603, 97310483508229643850787974274909946653, 49230006003690416105078802107081502212, 186624723026172141969101732162254984026, 180113569312430956460304045157917032883, 25320167879053189246084382330426816101, 223554333657580142329746061869814572341, 104725602674690613353967172484127150143, 194974656017203209835598782322195861206, 290171858038777269791118619687233212108, 205714217332446015440113025722313607497, 328829927715856240413784214611337153966, 228830113838219589603543242685918021737, 267621938517046584706647530012318892890, 328004975378179617076985202898867962050, 220346699090287156205224922125323321039, 178818084601337822373670352185192189314, 141189896737739026271207006176311050022, 295341689147671565873858610947682168478, 266772566706445399080725788477558783452, 145433164520227016488414725055974484346, 168865258393617881361087246946726055747, 120852017240255920066710921641421992239, 173126231889666509362919042928170490236]], missing_values_reference_list=['', '-', '?', nan])))
    preprocessor_features_categorical_steps.append(('boolean2float_transformer', autoai_libs.transformers.exportable.boolean2float(activate_flag=True)))
    preprocessor_features_categorical_steps.append(('cat_imputer', autoai_libs.transformers.exportable.CatImputer(activate_flag=True, missing_values=nan, sklearn_version_family='20', strategy='most_frequent')))
    preprocessor_features_categorical_steps.append(('cat_encoder', autoai_libs.transformers.exportable.CatEncoder(activate_flag=True, categories='auto', dtype=numpy.float64, encoding='ordinal', handle_unknown='error', sklearn_version_family='20')))
    preprocessor_features_categorical_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_categorical_ Pipeline
    preprocessor_features_categorical_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_categorical_steps)
    preprocessor_features_transformer_list.append(('categorical', preprocessor_features_categorical_pipeline))
    #
    # composing steps for preprocessor_features_numeric Pipeline
    #
    preprocessor_features_numeric__input_metadata = None
    preprocessor_features_numeric_steps = []
    preprocessor_features_numeric_steps.append(('num_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[13, 21])))
    preprocessor_features_numeric_steps.append(('num_floatstr2float_transformer', autoai_libs.transformers.exportable.FloatStr2Float(activate_flag=True, dtypes_list=['float_num', 'float_num'], missing_values_reference_list=[])))
    preprocessor_features_numeric_steps.append(('num_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_numeric_steps.append(('num_imputer', autoai_libs.transformers.exportable.NumImputer(activate_flag=True, missing_values=nan, strategy='median')))
    preprocessor_features_numeric_steps.append(('num_scaler', autoai_libs.transformers.exportable.OptStandardScaler(num_scaler_copy=None, num_scaler_with_mean=None, num_scaler_with_std=None, use_scaler_flag=False)))
    preprocessor_features_numeric_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_numeric_ Pipeline
    preprocessor_features_numeric_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_numeric_steps)
    preprocessor_features_transformer_list.append(('numeric', preprocessor_features_numeric_pipeline))
    # assembling preprocessor_features_ FeatureUnion
    preprocessor_features_pipeline = sklearn.pipeline.FeatureUnion(transformer_list=preprocessor_features_transformer_list)
    preprocessor_steps.append(('features', preprocessor_features_pipeline))
    preprocessor_steps.append(('permuter', autoai_libs.transformers.exportable.NumpyPermuteArray(axis=0, permutation_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 13, 21])))
    # assembling preprocessor_ Pipeline
    preprocessor_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_steps)
    steps.append(('preprocessor', preprocessor_pipeline))
    #
    # composing steps for cognito Pipeline
    #
    cognito__input_metadata = None
    cognito_steps = []
    cognito_steps.append(('0', autoai_libs.cognito.transforms.transform_utils.TA1(fun=numpy.square, name='square', datatypes=['numeric'], feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical], tgraph=None, apply_all=True, col_names=['Gender', 'College', 'In or Out of State', 'Citizenship', 'Marital Status', 'Type of High School', 'Applied Financial Aid', 'Received Financial Aid', 'Other Scholarships', 'Tuition Rate', 'Expected 4 Year Revenue', 'Open Houses', 'Visits', 'High School GPA', 'AP Tests Passed', 'Acceptance Type', 'Recruited Athlete', 'High School Class Size', 'HS Class Rank', 'Student Health', 'Total Class Hours', 'Overall GPA', 'Change in GPA', 'Housing', 'Club Memberships', 'Infirmary Visits', 'City', 'State'], col_dtypes=[dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32')], col_as_json_objects=None)))
    cognito_steps.append(('1', autoai_libs.cognito.transforms.transform_utils.FS1(cols_ids_must_keep=range(0, 28), additional_col_count_to_keep=20, ptype='classification')))
    cognito_steps.append(('2', autoai_libs.cognito.transforms.transform_utils.TAM(tans_class=autoai_libs.cognito.transforms.transform_extras.IsolationForestAnomaly, name='isoforestanomaly', tgraph=None, apply_all=True, col_names=['Gender', 'College', 'In or Out of State', 'Citizenship', 'Marital Status', 'Type of High School', 'Applied Financial Aid', 'Received Financial Aid', 'Other Scholarships', 'Tuition Rate', 'Expected 4 Year Revenue', 'Open Houses', 'Visits', 'High School GPA', 'AP Tests Passed', 'Acceptance Type', 'Recruited Athlete', 'High School Class Size', 'HS Class Rank', 'Student Health', 'Total Class Hours', 'Overall GPA', 'Change in GPA', 'Housing', 'Club Memberships', 'Infirmary Visits', 'City', 'State', 'square(High School GPA)', 'square(Overall GPA)', 'square(City)', 'square(State)'], col_dtypes=[dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32')], col_as_json_objects=None)))
    cognito_steps.append(('3', autoai_libs.cognito.transforms.transform_utils.FS1(cols_ids_must_keep=range(0, 28), additional_col_count_to_keep=20, ptype='classification')))
    cognito_steps.append(('4', autoai_libs.cognito.transforms.transform_utils.TAM(tans_class=sklearn.cluster.hierarchical.FeatureAgglomeration(affinity='euclidean', compute_full_tree='auto', connectivity=None, linkage='ward', memory=None, n_clusters=2, pooling_func=numpy.mean), name='featureagglomeration', tgraph=None, apply_all=True, col_names=['Gender', 'College', 'In or Out of State', 'Citizenship', 'Marital Status', 'Type of High School', 'Applied Financial Aid', 'Received Financial Aid', 'Other Scholarships', 'Tuition Rate', 'Expected 4 Year Revenue', 'Open Houses', 'Visits', 'High School GPA', 'AP Tests Passed', 'Acceptance Type', 'Recruited Athlete', 'High School Class Size', 'HS Class Rank', 'Student Health', 'Total Class Hours', 'Overall GPA', 'Change in GPA', 'Housing', 'Club Memberships', 'Infirmary Visits', 'City', 'State', 'square(High School GPA)', 'square(Overall GPA)', 'square(City)', 'square(State)', 'isoforestanomaly_0'], col_dtypes=[dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('int64')], col_as_json_objects=None)))
    cognito_steps.append(('5', autoai_libs.cognito.transforms.transform_utils.FS1(cols_ids_must_keep=range(0, 28), additional_col_count_to_keep=20, ptype='classification')))
    # assembling cognito_ Pipeline
    cognito_pipeline = sklearn.pipeline.Pipeline(steps=cognito_steps)
    steps.append(('cognito', cognito_pipeline))
    steps.append(('estimator', sklearn.ensemble.gradient_boosting.GradientBoostingClassifier(criterion='friedman_mse', init=None, learning_rate=0.1, loss='deviance', max_depth=5, max_features=0.986383771894106, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.019015242381366603, min_weight_fraction_leaf=0.0, n_estimators=59, n_iter_no_change=None, presort='auto', random_state=33, subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)))
    # assembling  Pipeline
    pipeline = sklearn.pipeline.Pipeline(steps=steps)
    return pipeline
pipeline = compose_pipeline()


re-attempting import of transform_extras from autoai_libs.cognito.transforms
re-attempting import of hierarchical from sklearn.cluster
re-attempting import of gradient_boosting from sklearn.ensemble
Pipeline successfully instantiated


/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.hierarchical module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.gradient_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


### 3. Extract needed parameter values from AutoAI run metadata

In [5]:

# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
#data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
if 'data_provenance' in _input_metadata:
    data_provenance = _input_metadata['data_provenance']
else:
    data_provenance = None
if 'pos_label' in _input_metadata and learning_type == 'classification':
    pos_label = _input_metadata['pos_label']
else:
    pos_label = None


### 4. Create dataframe from dataset in Cloud Object Storage

In [6]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_0 = {
    'ENDPOINT': 'unknown cloud service endpoint',
    'IBM_AUTH_ENDPOINT': 'unknown auth_endpoint',
    'APIKEY': 'unspecified api_key for bucket',
    'BUCKET': 'unspecified bucket name',
    'FILE': 'unspecified object name',
    'SERVICE_NAME': 'data_asset',
    'ASSET_ID': 'historical_student_rentention_data.csv',
    }


In [7]:
#  Read the data as a dataframe
import pandas as pd

csv_encodings=['UTF-8','Latin-1'] # supplement list of encodings as necessary for your data
df = None
readable = None  # if automatic detection fails, you can supply a filename here

# First, obtain a readable object
# Cloud Object Storage data access
# Assumes COS credentials are in a dictionary named 'credentials_0'
   
credentials = df = globals().get('credentials_0')       
if readable is None and credentials is not None :
    try:
        import types
        import pandas as pd
        import io
        import os
    except Exception as import_exception:
        print('Error with importing packages - check if you installed them on your environment')
    try:
        if credentials['SERVICE_NAME'] == 's3':
            try:
                from botocore.client import Config
                import ibm_boto3
            except Exception as import_exception:
                print('Installing required packages!')
                !pip install ibm-cos-sdk
                print('accessing data via Cloud Object Storage')
            try:
                cos_client = ibm_boto3.resource(service_name=credentials['SERVICE_NAME'],
                                                ibm_api_key_id=credentials['APIKEY'],
                                                ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                                                config=Config(signature_version='oauth'),
                                                endpoint_url=credentials['ENDPOINT'])
            except Exception as cos_exception:
                print('unable to create client for cloud object storage')
            try:
                cos_client.meta.client.download_file(Bucket=credentials['BUCKET'], Filename=credentials['FILE'], Key=credentials['FILE'])
            except Exception as cos_access_exception:
                print('unable to access data object in cloud object storage with credentials supplied') 
            try:
                df = pd.read_csv(credentials['FILE'],encoding = "ISO-8859-1")
                os.remove(credentials['FILE'])
            except Exception as cos_object_read_exception:
                print('unable to access data object from cos object with path supplied') 
        elif credentials['SERVICE_NAME'] == 'fs':
            print('accessing data via File System')
            try:
                df = pd.read_csv(credentials['FILE'])
            except Exception as FS_access_exception:
                print('unable to access data object in File System with path supplied') 
    except Exception as data_access_exception:
        print('unable to access data object with credentials supplied') 

# IBM Cloud Pak for Data data access
project_filename = globals().get('project_filename')       
if readable is None and 'credentials_0' in globals() and 'ASSET_ID' in credentials_0:
    project_filename = credentials_0['ASSET_ID']
if project_filename is not None:
    print('attempting project_lib access to ' + str(project_filename))
    try:
        from project_lib import Project
        project = Project.access()
        storage_credentials = project.get_storage_metadata()
        readable = project.get_file(project_filename)
    except Exception as project_exception:
        print('unable to access data using the project_lib interface and filename supplied')

        
# Use data_provenance as filename if other access mechanisms are unsuccessful
if readable is None and type(data_provenance) is str:
    print('attempting to access local file using path and name ' + data_provenance)
    readable = data_provenance

# Second, use pd.read_csv to read object, iterating over list of csv_encodings until successful
if readable is not None:
    for encoding in csv_encodings:
        try:
            df = pd.read_csv(readable, encoding=encoding)
            print('successfully loaded dataframe using encoding = ' + str(encoding))
            break
        except Exception as exception_csv:
            print('unable to read csv using encoding ' + str(encoding))
            print('handled error was ' + str(exception_csv))
    if df is None:
        print('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings))
        print(f'Please use \'insert to code\' on data panel to load dataframe.')
        raise(ValueError('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings)))

if isinstance(df,pd.DataFrame):
    print('Data loaded succesfully')


attempting project_lib access to historical_student_rentention_data.csv
unable to read csv using encoding UTF-8
handled error was Invalid file path or buffer object type: <class 'list'>
unable to read csv using encoding Latin-1
handled error was Invalid file path or buffer object type: <class 'list'>


### 5. Preprocess Data

In [8]:
# Drop rows whose target is not defined
target = target_label_name # your target name here
if learning_type == 'regression':
    df[target] = pd.to_numeric(df[target], errors='coerce')
df.dropna('rows', how='any', subset=[target], inplace=True)


AttributeError: 'dict' object has no attribute 'dropna'

In [9]:
# extract X and y
df_X = df.drop(columns=[target])
df_y = df[target]


AttributeError: 'dict' object has no attribute 'drop'

In [ ]:
# Detach preprocessing pipeline (which needs to see all training data)
preprocessor_index = -1
preprocessing_steps = [] 
for i, step in enumerate(pipeline.steps):
    preprocessing_steps.append(step)
    if step[0]=='preprocessor':
        preprocessor_index = i
        break
#if len(pipeline.steps) > preprocessor_index+1 and pipeline.steps[preprocessor_index + 1][0] == 'cognito':
    #preprocessor_index += 1
    #preprocessing_steps.append(pipeline.steps[preprocessor_index])
if preprocessor_index >= 0:
    preprocessing_pipeline = Pipeline(memory=pipeline.memory, steps=preprocessing_steps)
    pipeline = Pipeline(steps=pipeline.steps[preprocessor_index+1:])

In [ ]:
# Preprocess X
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
known_values_list.clear()  #  known_values_list is filled in by the preprocessing_pipeline if needed
preprocessing_pipeline.fit(df_X.values, df_y.values)
X_prep = preprocessing_pipeline.transform(df_X.values)

### 6. Split data into Training and Holdout sets

In [ ]:
# determine learning_type and perform holdout split (stratify conditionally)
if learning_type is None:
    # When the problem type is not available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
    # Caution:  This can mis-classify regression targets that can be expressed as integers as multiclass, in which case manually override the learning_type
    from sklearn.utils.multiclass import type_of_target
    if type_of_target(df_y.values) in ['multiclass', 'binary']:
        learning_type = 'classification'
    else:
        learning_type = 'regression'
    print('learning_type determined by type_of_target as:',learning_type)
else:
    print('learning_type specified as:',learning_type)
    
from sklearn.model_selection import train_test_split
if learning_type == 'classification':
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state)


#### 7. Generate features via Feature Engineering pipeline

In [ ]:
#Detach Feature Engineering pipeline if next, fit it, and transform the training data
fe_pipeline = None
if pipeline.steps[0][0] == 'cognito':
    try:
        fe_pipeline = Pipeline(steps=[pipeline.steps[0]])
        X = fe_pipeline.fit_transform(X, y)
        X_holdout = fe_pipeline.transform(X_holdout)
        pipeline.steps = pipeline.steps[1:]
    except IndexError:
        try:
            print('Trying to compose pipeline with some of cognito steps')
            fe_pipeline = Pipeline(steps = list([pipeline.steps[0][1].steps[0],pipeline.steps[0][1].steps[1]]))
            X = fe_pipeline.fit_transform(X, y)
            X_holdout = fe_pipeline.transform(X_holdout)
            pipeline.steps = pipeline.steps[1:]
        except IndexError:
            print('Composing pipeline without cognito steps!')
            pipeline.steps = pipeline.steps[1:]


 ### 8. Additional setup: Define a function that returns a scorer for the target's positive label

In [ ]:
# create a function to produce a scorer for a given positive label
def make_pos_label_scorer(scorer, pos_label):
    kwargs = {'pos_label':pos_label}
    for prop in ['needs_proba', 'needs_threshold']:
        if prop+'=True' in scorer._factory_args():
            kwargs[prop] = True
    if scorer._sign == -1:
        kwargs['greater_is_better'] = False
    from sklearn.metrics import make_scorer
    scorer=make_scorer(scorer._score_func, **kwargs)
    return scorer

### 9. Fit pipeline, predict on Holdout set, calculate score, perform cross-validation

In [ ]:
# fit the remainder of the pipeline on the training data
pipeline.fit(X,y)

In [ ]:
# predict on the holdout data
y_pred = pipeline.predict(X_holdout)

In [ ]:
# compute score for the optimization metric
# scorer may need pos_label, but not all scorers take pos_label parameter
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
score = None
#score = scorer(pipeline, X_holdout, y_holdout)  # this would suffice for simple cases
pos_label = None  # if you want to supply the pos_label, specify it here
if pos_label is None and 'pos_label' in _input_metadata:
    pos_label=_input_metadata['pos_label']
try:
    score = scorer(pipeline, X_holdout, y_holdout)
except Exception as e1:
    if pos_label is None or str(pos_label)=='':
        print('You may have to provide a value for pos_label in order for a score to be calculated.')
        raise(e1)
    else:
        exception_string=str(e1)
        if 'pos_label' in exception_string:
            try:
                scorer = make_pos_label_scorer(scorer, pos_label=pos_label)
                score = scorer(pipeline, X_holdout, y_holdout)
                print('Retry was successful with pos_label supplied to scorer')
            except Exception as e2:
                print('Initial attempt to use scorer failed.  Exception was:')
                print(e1)
                print('')
                print('Retry with pos_label failed.  Exception was:')
                print(e2)
        else:
            raise(e1)

if score is not None:
    print(score)

In [ ]:
# cross_validate pipeline using training data
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold
if learning_type == 'classification':
    fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
else:
    fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
cv_results = cross_validate(pipeline, X, y, cv=fold_generator, scoring={optimization_metric:scorer}, return_train_score=True)
import numpy as np
np.mean(cv_results['test_' + optimization_metric])

In [ ]:
cv_results